# Alternate Class Instantiation

Python 3 has added type annotations / hinting to the standard library and core language syntax. The implementation of this extended `__getitem__` to [the class level](), exposing an interesting mechanism to _spawn variants_. Having written "alternate universe" tempate engines which "abuse" Python language semantics, including `__getitem__`, to perform work they were never intended to… I see an opportunity.

In [Marrow Tags](https://github.com/marrow/tags#readme) dict-like dereferencing is used to denote child object containment, not member retrieval. For example, a simple "template":

```python
html [
        head [ title [ "Welcome!", " — ", SITE_NAME ] ],
        flush,
        body ( class_ = 'nav-welcome' ) [
                site_header(),
                p [ "Lorem ipsum dolor sit amet…" ],
                site_footer()
            ]
    ]
```

This somewhat demonstrates that Python is not, in fact, a "whitespace sensitive" language. Indentation within a derefernece (or any structural literal) is ignored. However, this poses a problem. Where is `html`, or `head`, or `title` coming from? Can their behaviors be customized or adapted?

Marrow Tags populates a default set from the HTML specification of the time but _custom tags_ are becoming a more popular feature to utilize. The web has come a long way, and there are [clearly documented API specifications available](https://developer.mozilla.org/en-US/docs/Web/API/Element) outlining how to manipulate, query, and interrogate these types of objects. Some tags may desire additional functionality: a `<head>` element, for example, need not be emitted if the first child is a `<title>`. Similar with the `<body>` element if the first child is a content element, or `<html>` itself if no attributes are present. [Really!](https://gist.github.com/amcgregor/71c62ea2984839a9063232ed2c0adf27)

As there are several components to this problem and its solution, we'll start by defining a basic "HTML tag" representation which will largely follow the [published specification](https://dom.spec.whatwg.org/#interface-element) on attribute naming and represent our overall data model.

## Basic Tag

The initial collection of imports provide the primitives used to form [type annotations]() and [type hinting]() which can be optionally enforced statically using tools like [mypy](), but also at runtime using libraries such as [typeguard]().

We also define one utility function, useful later.

In [1]:
from collections.abc import Collection
from textwrap import indent
from typing import Any, Dict, Iterable, List, Optional, Set, Type, TypeVar, Union
from re import compile as re

camel = lambda s: (m.group(0).lower() for m in camel.pattern.finditer(s))
camel.pattern = re('.+?(?:(?<=[a-z])(?=[A-Z])|(?<=[A-Z])(?=[A-Z][a-z])|$)')

In [2]:
'-'.join(camel('myAwesomeTag'))

'my-awesome-tag'

With the basics needed for self-documentation out of the way, we'll form the basic definition of a Tag. We won't worry about specialized behavior using this new functionality yet.

In [3]:
from xml.sax.saxutils import quoteattr  # Used when rendering element attributes.

class T:
    children: List
    classList: Set[str]
    localName: str
    attributes: dict
    
    def __init__(self, name:str, children:Optional[List]=None, **kw) -> None:
        self.children = children or []  # Populate empty defaults.
        self.classList = set()
        self.attributes = {'class': 'classList'}
        
        self.localName = name
        
        for name, value in kw.items():
            setattr(self, name, value)
    
    def __repr__(self) -> str:
        return f"<tag '{self.localName}' children={len(self)}{' ' if self.attributes else ''}{self.attributeMapping}>"
    
    def __len__(self):
        """Our length is that of the number of our child elements."""
        return len(self.children)
    
    def __iter__(self):
        """Act as if we are our collection of children when iterated."""
        return iter(self.children)
    
    def __str__(self) -> str:
        parts = []  # These are the string fragments that will ultimately be returned as one.
        
        parts.extend(('<', self.localName))
        
        for key, value in sorted(self.attributeMapping.items()):
            if key[0] == '_': continue  # Armour against protected attribute access.
            
            # Skip values that are non-zero falsy, working around the fact that False == 0.
            if not value and (value is False or value != 0):
                continue
            
            name = str(key).rstrip('_').replace('__', ':').replace('_', '-')
            
            # Add spacer if needed.
            if len(parts) == 2:
                parts.append(' ')

            if value is True:  # For explicitly True values, don't emit a value for the attribute.
                parts.append(name)
                continue

            # Non-string iterables (such as lists, sets, tuples, etc.) are treated as space-separated strings.
            if isinstance(value, Iterable) and not isinstance(value, str):
                value = " ".join(str(i) for i in value)
            
            value = quoteattr(str(value))
            if " " not in value:
                value = value.strip('"')
            
            parts.extend((name, "=", value))

        parts.append('>')
        if self.children:
            if __debug__:  # Prettier "linted" output when optimizations aren't enabled.
                parts.append("\n")
                parts.append(indent("".join(str(child) for child in self), "\t"))
            else:
                parts.extend(str(child) for child in self)
        parts.extend(('\n</' if __debug__ else '</', self.localName, '>\n' if __debug__ else '>'))
        
        return ''.join(parts)
        
        # Three different possible "null" / "empty" scenarios.
        return f'<{self.localName}></{self.localName}>' + "\n" if __debug__ else ""  # Missing contents.
        return f'<{self.localName} />'  # XML-like explicit NULL element.
        return f'<{self.localName}>'  # HTML5-like self-closing tag.
    
    def __len__(self):
        return len(self.children)
    
    @property
    def attributeMapping(self):
        return {k: v for k, v in {name: getattr(self, origin, None) for name, origin in self.attributes.items()}.items() if v}
    
    # API-conformant aliases for "localName".
    tagName = \
    nodeName = \
        property(lambda self: self.localName)

In [4]:
print(T)
print(repr(T('html')), T('html'), sep="\t")
print(repr(T('p')), T('p'), sep="\t")

ex = T('p', ['Lorem ipsum dolor...'], classList={'example'})
print(repr(ex), ex, sep="\n")
print(ex.attributeMapping)

page = T('html', [
        T('head', [
            T('title', ["Welcome"])
        ]),
        T('body', [
            T('p', ["Lorem ipsum dolor sit amet…"])
        ]),
    ])

print("", repr(page), "", page, sep="\n")

<class '__main__.T'>
<tag 'html' children=0 {}>	<html>
</html>

<tag 'p' children=0 {}>	<p>
</p>

<tag 'p' children=1 {'class': {'example'}}>
<p class=example>
	Lorem ipsum dolor...
</p>

{'class': {'example'}}

<tag 'html' children=2 {}>

<html>
	<head>
		<title>
			Welcome
		</title>

	</head>
	<body>
		<p>
			Lorem ipsum dolor sit amet…
		</p>

	</body>

</html>



It may be noticeable that there are some optimizations we can apply to the HTML generation, and by gum this is an ugly syntax to try to work with. We can improve both of these aspects. Users with a Perl background may be more comfortable with this syntax than others.

We'll deal with the HTML serializaiton later.

## The Tag Factory

A key point to these elements is that _instantiation_ creates a new element _instance_, and that there isn't a distinct class per element. Writing your HTML this way would be highly cumbersome. Having to write out not just that it is a tag, but which one, its exact children, etc., up-front is sub-optimal. We could provide dedicated subclasses for every possible element, using the module scope to contain them, but there is a better, more dynamic way.

Instances have the ability to participate in the attribute lookup protocol using `__getattr__` and `__getattribute__` methods. So too do _metaclasses_ have the ability to interpose attribute lookup on _classes_.

Step one would then be to define a base metaclass that our "magical" classes will inherit from to provide compile–time and run–time behavior.

In [5]:
class TagMeta(type):
    def __new__(meta, name, bases, attrs):
        cls = type.__new__(meta, str(name), bases, attrs)
        return cls
    
    def __getattr__(Tag, name:str):
        return Tag('-'.join(camel(name)))


class Tag(T, metaclass=TagMeta): ...

In [6]:
print(repr(Tag))
print(repr(Tag.title))
print(Tag.title)

<class '__main__.Tag'>
<tag 'title' children=0 {}>
<title>
</title>



Congratulations, we now have the class itself as a factory for its own instances. Which sounds absolutely unimpressive, as that is the purpose of classes, however the factory is _attribute access_ (to otherwise unknown attributes) **itself**, not invocation. The name of the tag is inferred from the name of the attribute. This even supports "custom tags" using `camelCase` notation.

In [7]:
print(Tag.fileUpload)

<file-upload>
</file-upload>



All tags are treated the same, in this rough draft. There are differences, especially in making the serializaiton attractive to humans such as the distinction between _block_ and _inline_ elements, which also offer points for minor optimization.

Notice, however, that absolutely nothing is currently HTML-specific!

## Mutation

What if we now wish to provide new values for attributes? Attribute access of the class is returning an already-instantiated value, making it too late to provide values to `__init__`. Instances can be made _callable_ by defining a `__call__` method, however, mimmicing use of the instance as if it were a class. In the "assigning children" case, a fresh-off-the-press newly minted instance will never have children, so mutation of the existing instance and returning the same (`self`) is a highly practical solution.

Note in the `__call__` case, returning what is technically a new instance, with all existing attributes transferred and optionally mutated, is preferable. This has the side-effect that one can create "template" objects, but it is critical to be aware that _mutable_ objects, such as lists, sets, etc., will have their contents shared amongst the derived copies; mutation of one will appear to alter them all.

In [8]:
class Tag(T, metaclass=TagMeta):
    def __call__(self, **attributes) -> T:
        """Produce a new, cloned and mutated instance of this tag incorporating attribute changes."""
        instance = self.__class__(self.localName, self.children)  # Mutant pools children!
        instance.__dict__ = self.__dict__.copy()  # It pools all mutable attributes!
        
        for name, value in attributes.items():
            setattr(instance, name, value)
        
        return instance
    
    def __getitem__(self, children) -> T:
        """Mutate this instance to add these children, returning this instance for chained manipulation."""
        if isinstance(children, (tuple, list)):
            self.children.extend(children)
        else:
            self.children.append(children)
        
        return self

In [9]:
bdole = Tag.p(classList={'name'})["Bob Dole"]
print(bdole, bdole.__dict__, "", sep="\n")
print(Tag.p(classList={'fancy', 'wau'})["Much content, super elide."])

<p class=name>
	Bob Dole
</p>

{'children': ['Bob Dole'], 'classList': {'name'}, 'attributes': {'class': 'classList'}, 'localName': 'p'}

<p class="fancy wau">
	Much content, super elide.
</p>



You might not have noticed it earlier, but see how the `class` attribute of the first paragraph has no quotes.

We can now replicate an earlier example using a much nicer syntax.

In [10]:
page = Tag.html [
        Tag.head [
            Tag.title ["Welcome"]
        ],
        Tag.body [
            Tag.p ["Lorem ipsum dolor sit amet…"]
        ],
    ]

print(repr(page), page, sep="\n\n")

<tag 'html' children=2 {}>

<html>
	<head>
		<title>
			Welcome
		</title>

	</head>
	<body>
		<p>
			Lorem ipsum dolor sit amet…
		</p>

	</body>

</html>



Returning to a point touched on earlier, there's nothing HTML-specific about any of this. Let's see what happens attempting to generate another type of document structured using tags…

In [11]:
feed = Tag.rss[Tag.channel[
    Tag.title["My awesome RSS feed!"],
    Tag.link["https://example.com/"],
    Tag.item["..."]
]]

print(feed)

<rss>
	<channel>
		<title>
			My awesome RSS feed!
		</title>
		<link>
			https://example.com/
		</link>
		<item>
			...
		</item>

	</channel>

</rss>



Technically what we're doing here is constructing a _heirarchical tagged string_ representation. Like a word processor, just with complete freedom as to the tags in use.

## Customization

In order to permit subclassing of Tag to implement specific customizations to tag behavior, the metaclass will require some changes to how it determines which class to instantiate when an unknown class-level attribute is accessed. Lucky for us, a class method is provided to identify these to us. Unfortunately, it is not a mapping (how could it be?) but we can reasonably mandate that a given tag have only one implementation.

In [12]:
class HTML(Tag):
    ...

Tag.__subclasses__()

[__main__.HTML]

In [13]:
class TagMeta(type):
    def __new__(meta, name, bases, attrs):
        cls = type.__new__(meta, str(name), bases, attrs)
        return cls
    
    @property
    def __subclass_map__(Tag):
        return {subclass.__name__: subclass for subclass in Tag.__subclasses__()}
    
    def __getattr__(Tag, name:str):
        localName = '-'.join(camel(name))
        name = name[0].upper() + name[1:]
        
        Tag = Tag.__subclass_map__.get(name, Tag)
        
        return Tag(localName)


class Tag(T, metaclass=TagMeta):
    def __call__(self, **attributes) -> T:
        """Produce a new, cloned and mutated instance of this tag incorporating attribute changes."""
        instance = self.__class__(self.localName, self.children)  # Mutant pools children!
        instance.__dict__ = self.__dict__.copy()  # It pools all mutable attributes!
        
        for name, value in attributes.items():
            setattr(instance, name, value)
        
        return instance
    
    def __getitem__(self, children) -> T:
        """Mutate this instance to add these children, returning this instance for chained manipulation."""
        if isinstance(children, (tuple, list)):
            self.children.extend(children)
        else:
            self.children.append(children)
        
        return self


class _Prefixed:
    prefix: str
    
    def __str__(self):
        return self.prefix + ("\n" if __debug__ else "") + super().__str__()


class _Elidable:
    def __str__(self):
        if not self.classList:
            return "".join(str(child) for child in self)
        
        if __debug__:  # Prettier "linted" output when optimizations aren't enabled.
            parts.append("\n")
            parts.append(indent("".join(str(child) for child in self), "\t"))
        else:
            parts.extend(str(child) for child in self)


class Html(_Prefixed, _Elidable, Tag):
    prefix = "<!DOCTYPE html>"

class Head(_Elidable, Tag): pass
class Body(_Elidable, Tag): pass

Tag.__subclass_map__

{'Html': __main__.Html, 'Head': __main__.Head, 'Body': __main__.Body}

In [14]:
Tag.html.__class__

__main__.Html

In [15]:
Tag.title.__class__

__main__.Tag

In [16]:
print(Tag.html[Tag.title["Hello world!"]])

<!DOCTYPE html>
<title>
	Hello world!
</title>



In [17]:
page = Tag.html [
        Tag.head [
            Tag.title ["Welcome"]
        ],
        Tag.body [
            Tag.p ["This is a complete and fully-formed HTML document."]
        ],
    ]

print(page)

<!DOCTYPE html>
<title>
	Welcome
</title>
<p>
	This is a complete and fully-formed HTML document.
</p>

